# Algoritmo Genético

In [12]:
import random

# peso máximo da mochila
limite_peso = 120 #C
tamanho_populacao = 10 #Np
probabilidade_mutacao = 0.2 # Pm
probabilidade_crossover = 0.8 #Pc
maximo_geracoes = 500 

# população inicial do problema
itens_disponiveis = [ 
    { 'peso': 3, 'valor': 1},
    { 'peso': 8, 'valor': 3},
    { 'peso': 12, 'valor': 1},
    { 'peso': 2, 'valor': 8},
    { 'peso': 8, 'valor': 9},
    { 'peso': 4, 'valor': 3},
    { 'peso': 4, 'valor': 2},
    { 'peso': 5, 'valor': 8},
    { 'peso': 1, 'valor': 5},
    { 'peso': 1, 'valor': 1},
    { 'peso': 8, 'valor': 1},
    { 'peso': 6, 'valor': 6},
    { 'peso': 4, 'valor': 3},
    { 'peso': 3, 'valor': 2},
    { 'peso': 3, 'valor': 5},
    { 'peso': 5, 'valor': 2},
    { 'peso': 7, 'valor': 3},
    { 'peso': 3, 'valor': 8},
    { 'peso': 5, 'valor': 9},
    { 'peso': 7, 'valor': 3},
    { 'peso': 4, 'valor': 2},
    { 'peso': 3, 'valor': 4},
    { 'peso': 7, 'valor': 5},
    { 'peso': 2, 'valor': 4},
    { 'peso': 3, 'valor': 3},
    { 'peso': 5, 'valor': 1},
    { 'peso': 4, 'valor': 3},
    { 'peso': 3, 'valor': 2},
    { 'peso': 7, 'valor': 14},
    { 'peso': 19, 'valor': 32},
    { 'peso': 20, 'valor': 20},
    { 'peso': 21, 'valor': 19},
    { 'peso': 11, 'valor': 15},
    { 'peso': 24, 'valor': 37},
    { 'peso': 13, 'valor': 18},
    { 'peso': 17, 'valor': 13},
    { 'peso': 18, 'valor': 19},
    { 'peso': 6, 'valor': 10},
    { 'peso': 15, 'valor': 15},
    { 'peso': 25, 'valor': 40},
    { 'peso': 12, 'valor': 17},
    { 'peso': 19, 'valor': 39},
]


In [76]:
def gerar_populacao_inicial():
    num_max_itens = len(itens_disponiveis)
    # return [random.choice([0,1]) for i in range(num_max_itens)]
    return [ [random.choice([0,1]) for i in range(num_max_itens)] for j in range(tamanho_populacao)] 


def print_geracao(geracao):
    for g in geracao:
        print('\n')
        for gene in g:
            print('%s ' %(gene), end='', flush=True)
        

populacao_inicial = gerar_populacao_inicial()

print('População inicial')
print_geracao(populacao_inicial)

População inicial


0 1 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 0 1 0 0 1 1 1 1 1 

0 1 0 0 1 1 1 0 1 1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 1 0 0 1 0 1 1 0 0 1 1 0 1 0 0 

1 1 0 1 0 1 1 1 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 0 

0 0 0 0 1 1 1 0 0 1 0 1 1 0 0 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 0 0 1 

0 1 0 1 1 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 1 0 1 1 

1 0 0 1 0 1 0 1 1 1 0 0 0 0 0 1 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 

1 1 1 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 1 0 0 1 1 0 0 1 1 1 0 1 1 0 1 

1 1 0 1 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 0 0 

0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 0 1 

0 0 0 1 0 0 1 1 0 1 1 0 1 1 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 

In [85]:
# recupera os indices dos valores selecionados para serem inseridos na mochila
def get_index_populacao(populacao):
    return [[index for (index, item) in enumerate(itens_disponiveis) if p[index]] for p in populacao]

# recupera os itens selecionados da população
def get_itens_populacao(populacao):
    return [[item for (index, item) in enumerate(itens_disponiveis) if p[index]] for p in populacao]

# recupera os itens de um unico individuo da população
def get_itens_individuo(individuo):
    return [item for (index, item) in enumerate(itens_disponiveis) if individuo[index]]

# Função fitness para uma população
# Retorna a soma dos pesos e valores de cada item da mochila
def fitness(populacao):
    itens_populacao = get_itens_populacao(populacao)
    
    fitness_values = []
    for itens_individuo in itens_populacao:
        fitness_values.append( sum_itens(itens_individuo) )
        
    return fitness_values

# Função fitness para um unico indivíduo da população
def fitness_individuo(individuo):
    itens_individuo = get_itens_individuo(individuo)
    
    return sum_itens(itens_individuo)

# soma os pesos e valores dos itens apresentados
def sum_itens(itens):
    soma_peso = 0
    soma_valor = 0
    for item in itens:
        soma_peso += item['peso']
        soma_valor += item['valor']
    
    return (soma_peso, soma_valor)

print(fitness(populacao_inicial))

print(fitness_individuo(populacao_inicial[5]))

[(208, 252), (175, 200), (149, 177), (143, 150), (205, 257), (113, 136), (248, 267), (169, 185), (249, 306), (104, 102)]
(113, 136)
